In [270]:
import pandas as pd
from tabulate import tabulate
from decimal import Decimal
from re import sub
import numpy as np
import dateparser

In [278]:
frDf = pd.read_csv("assets_fundraise.csv")
frDf.rename(columns = {
    'Total Raised':'Total Raised ($)',
    'R1 Price':'R1 Price ($)', 
    'R2 Price':'R2 Price ($)',
    'R3 Price':'R3 Price ($)',
    'R4 Price':'R4 Price ($)',
    'R5 Price':'R5 Price ($)',
    'R1 Raise':'R1 Raise ($)', 
    'R2 Raise':'R2 Raise ($)',
    'R3 Raise':'R3 Raise ($)',
    'R4 Raise':'R4 Raise ($)',
    'R5 Raise':'R5 Raise ($)',
}, inplace = True)


for index in frDf.index:
    
    # for each round
    for r in range(1,6):
        
        # normalize price values to USD and remove $
        try:
            frDf.at[index, f"R{r} Price ($)"] = Decimal(sub(r'[^\d.]', '', frDf.at[index, f"R{r} Price ($)"]))
        except:
            pass
        
        # normalize raise values to USD and remove $
        try:
            frDf.at[index, f"R{r} Raise ($)"] = Decimal(sub(r'[^\d.]', '', frDf.at[index, f"R{r} Raise ($)"]))
        except:
            pass

        # normalize date 
        try:
            t = dateparser.parse(frDf.at[index, f"R{r} date"])
            frDf.at[index, f"R{r} date"] = dateparser.parse(frDf.at[index, f"R{r} date"])
        except:
            pass
        

        

/Users/shohihara/Developer/shiftbase/UNCHAIN/data_and_research/messari_dataset/venv/lib/python3.9/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [283]:
assetDf = pd.read_csv("assets.csv")
for index in frDf.index:
    try:
        assetName = (frDf.at[index, 'asset']).lower().replace('-', ' ')
        assetType = assetDf.loc[assetDf['name'].str.lower() == assetName, 'token__token_type'].values
        if len(assetType) == 0:
            pass
        else:
            assetType = assetType[0].split(",")[0]
            # if native token
            if assetType == 'Native':
                total_supply = frDf.at[index, 'Total supply']
                total_raised = frDf.at[index, 'Total Raised ($)']
                privateRaises = []
                privateIssues = []
                roundRaises = []
                roundIssues = []
                for r in range(1,6):
                    raiseValue = float(frDf.at[index, f"R{r} Raise ($)"])
                    roundTokenPrice = float(frDf.at[index, f"R{r} Price ($)"])
                    roundTokenIssue = float(frDf.at[index, f"R{r} Tokens For Sale"].replace(',',''))
                    if raiseValue == np.nan:
                        raiseValue = 0
                    if roundTokenPrice == np.nan:
                        roundTokenPrice = 0
                    if roundTokenIssue == np.nan:
                        roundTokenIssue = 0

                    roundRaises.append(roundTokenPrice * roundTokenIssue)
                    roundIssues.append(roundTokenIssue)
                    # print(assetName, raiseValue, roundTokenPrice, roundTokenIssue)

                    if frDf.at[index, f"R{r} type"] not in ["ICO ", "IDO", "IEO ", np.nan]:
                        # round_type = frDf.at[index, f"R{r} type"]
                        privateRaises.append(roundTokenPrice * roundTokenIssue)
                        privateIssues.append(roundTokenIssue)
                    else:
                        pass
                
                # print(privateRaises, roundRaises)
                raise_ratio = sum(privateRaises) / sum(roundRaises)
                print(assetName, "raise amount (private) / total raised =", raise_ratio)
                
                #pd.DataFrame({'Issues': roundIssues}, index=[f"R{r}" for r in range(len(roundIssues))] ).plot.pie(y="Issues", figsize=(6,6))
                
            elif assetType[:6] == 'ERC20':
                pass
            else:
                pass

    except:
        pass

solana raise amount (private) / total raised = 0.9330149003786942
avalanche raise amount (private) / total raised = 0.2202438600412228
